# Refactor EV

In [1]:
import itertools
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import OrderedDict

import pprint


In [2]:
def safe_div(x, y):
    '''
    Safe division, return 0 if y is 0.

    Parameters
    ----------
    x: float
        numerator.
    y: float
        denominator.
    '''
    if y == 0:
        return 0
    else:
        return x/y

In [3]:
class DictAttr():

    def __init__(self, attr):
        """
        Base class for attribtues stored in OrderedDict

        Parameters
        ----------
        attr: OrderedDict
            Data attribute dictionary
        """
        for key, val in attr.items():
            setattr(self, key, val)
        self._dict = self.as_dict()

    def as_dict(self) -> OrderedDict:
        """
        Return the config fields and values in an ``OrderedDict``.
        """
        out = []
        for key, val in self.__dict__.items():
            if not key.startswith('_'):
                out.append((key, val))
        return OrderedDict(out)

    def __repr__(self):
        return pprint.pformat(self._dict)


In [44]:
class EVData():
    """EV data class"""

    def __init__(self, N, col, idx) -> None:
        self.v = -1 * np.ones((N, len(col)))
        self._col = col
        self._idx = idx

    # def __repr__(self):
    #     return self.as_df(self._col)

    def as_df(self):
        """
        Return data as pandas DataFrame

        Parameters
        ----------
        col: list
            Column names.
        """
        df = pd.DataFrame(self.v, columns=self._col)
        df['idx'] = self._idx
        return df[['idx'] + self._col]


class EVStation():
    """
    EV Station class, holds EV data, control EV status, and collecte EV info.
    """

    def __init__(self, config, ud_param, nd_param, name='EVS') -> None:
        """
        Parameters
        ----------
        config: DictAttr
            EV station configuration.
        ud_param: Dict of Dict
            Uniform distribution parameters.
        nd_param: Dict of Dict
            Normal distribution parameters.

        config
        ------
        N: number of EVs
        Ns: number of SOC intervals
        step: simulation step size
        seed: random seed

        nd_param
        --------
        soci: initial SOC
        socd: demanded SOC
        ts1: start charging time 1
        ts2: start charging time 2
        tf1: finish charging time 1
        tf2: finish charging time 2
        tt: tolerance of increased charging time

        ud_param
        --------
        Pc: charging power
        Pd: discharging power
        nc: charging efficiency
        nd: discharging efficiency
        Q: battery capacity
        """
        self.name = name
        self.config = DictAttr(config)
        idx = [f'{self.name}_EV{i}' for i in range(self.config.N)]
        dcols = {'s': ['soci', 'socd', 'Pc', 'Pd', 'nc', 'nd',
                       'Q', 'ts', 'tf', 'tt', 'soc0', 'na0'],
                 'd': ['u', 'soc', 'c', 'sx', 'na', 'agc', 'lc', 'mod'],
                 }
        self.evd = EVData(N=self.config.N, col=dcols['s'], idx=idx)  # static data
        self.ev = EVData(N=self.config.N, col=dcols['d'], idx=idx)  # dynamic data

        # --- 1. uniform distribution parameters ---
        ud_cols = ['Pc', 'Pd', 'nc', 'nd', 'Q']
        np.random.seed(self.config.seed)
        for col in ud_cols:
            lb = ud_param[col]['lb']
            ub = ud_param[col]['ub']
            self.evd.v[:, dcols['s'].index(col)] = np.random.uniform(
                low=ud_param[col]['lb'],
                high=ud_param[col]['ub'],
                size=self.config.N)
        # NOTE: assumtpion: nc = nd
        self.evd.v[:, dcols['s'].index('nc')] = self.evd.v[:, dcols['s'].index('nd')]

        # --- 2. normal distribution parameters ---
        # --- 2.1 ---
        nd_cols = ['soci', 'socd', 'tt']
        for col in nd_cols:
            if col in dcols['s']:
                data_array = self.evd.v
                sord = 's'
            elif col in dcols['d']:
                data_array = self.ev.v
                sord = 'd'
            else:
                raise ValueError('Column name not found.')
            a = (nd_param[col]['lb'] - nd_param[col]['mu']) / nd_param[col]['var']
            b = (nd_param[col]['ub'] - nd_param[col]['mu']) / nd_param[col]['var']
            distribution = stats.truncnorm(a, b, loc=nd_param[col]['mu'], scale=nd_param[col]['var'])
            data_array[:, dcols[sord].index(col)] = distribution.rvs(self.config.N,
                                                                     random_state=self.config.seed)

        # --- 2.2 time parameters ---




    def rctrl(self):
        """Response to control signal"""
        pass

    def run(self):
        """Run simulation"""
        pass


evs_config = {'N': 100, 'Ns': 20, 'step': 1, 'seed': 2022}
nd_param = {'soci': {'mu': 0.3, 'var': 0.05, 'lb': 0.2, 'ub': 0.4},
            'socd': {'mu': 0.8, 'var': 0.03, 'lb': 0.7, 'ub': 0.9},
            'ts1': {'mu': -6.5, 'var': 3.4, 'lb': 0.0, 'ub': 5.5},
            'ts2': {'mu': 17.5, 'var': 3.4, 'lb': 5.5, 'ub': 24.0},
            'tf1': {'mu': 8.9, 'var': 3.4, 'lb': 0.0, 'ub': 20.9},
            'tf2': {'mu': 32.9, 'var': 3.4, 'lb': 20.9, 'ub': 24.0},
            'tt': {'mu': 0.5, 'var': 0.02, 'lb': 0, 'ub': 1},
            }
ud_param = {'Pc': {'lb': 5.0, 'ub': 7.0},
            'Pd': {'lb': 5.0, 'ub': 7.0},
            'nc': {'lb': 0.88, 'ub': 0.95},
            'nd': {'lb': 0.88, 'ub': 0.95},
            'Q': {'lb': 20.0, 'ub': 30.0},
            }

evs = EVStation(config=evs_config, ud_param=ud_param, nd_param=nd_param, name='EVS')
evs.evd.as_df()


,idx,soci,socd,Pc,Pd,nc,nd,Q,ts,tf,tt,soc0,na0
0,EVS_EV0,0.207169,0.729959,5.018717,5.897818,0.912224,0.912224,24.284339,-1.0,-1.0,0.452978,-1.0,-1.0
1,EVS_EV1,0.299887,0.799929,5.998116,6.019184,0.896877,0.896877,25.149405,-1.0,-1.0,0.499953,-1.0,-1.0
2,EVS_EV2,0.243910,0.763790,5.226767,6.549000,0.937510,0.937510,20.695047,-1.0,-1.0,0.475825,-1.0,-1.0
3,EVS_EV3,0.226378,0.750759,5.099948,5.569627,0.939463,0.939463,21.000653,-1.0,-1.0,0.467098,-1.0,-1.0
4,EVS_EV4,0.322962,0.814473,6.370815,5.156710,0.945409,0.945409,26.365037,-1.0,-1.0,0.509657,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,EVS_EV95,0.351760,0.833240,6.732769,6.195102,0.940405,0.940405,21.434957,-1.0,-1.0,0.522189,-1.0,-1.0
96,EVS_EV96,0.323107,0.814565,6.372997,5.482221,0.925943,0.925943,26.723112,-1.0,-1.0,0.509719,-1.0,-1.0
97,EVS_EV97,0.213386,0.738495,5.039523,5.075954,0.942757,0.942757,23.887057,-1.0,-1.0,0.458826,-1.0,-1.0
98,EVS_EV98,0.234121,0.756777,5.148923,6.832513,0.908200,0.908200,28.592836,-1.0,-1.0,0.471133,-1.0,-1.0


In [7]:
01.0

1.0

In [6]:
evs.config.N

1000

In [ ]:
class EVCenter():
    """EV control center, based on state space model"""
    def __init__(self) -> None:
        super().__init__()

    def setup(self):
        """Setup State Space Model"""
        pass

    def efrc(self):
        """Estimate frequency regulation capacity"""
        pass

    def gctrl(self):
        """Generte control signal"""
        pass

In [2]:
# --- Initialization ---
evs1 = EVStation()  # EV Station
evs2 = EVStation()
evc = EVCenter()  # EV Center

# --- Dispatch ---
evc.efrc()  # EVS estimate FRC
evc.send()  # EVS send info to transmisstion

# --- DG Control ---
evc.receive()  # receive info from transmission
evc.gctrl()  # generate control signal
evc.send()  # send control signal to EVStation

evs1.receive()  # receive control signal
evs1.rctrl()  # response to control signal

evs2.receive()  # receive control signal
evs2.rctrl()  # response to control signal

# --- Run ---
# Run Distribution
fd.run()

# RUN DG
evs1.run()
evs2.run()
# Federate Power with Transmission

# RUN Dynamic
tc.run()
